In [3]:
import os
import sys
import time
import warnings
from random import shuffle

import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
import numpy as np
import keras
from keras import backend as K
import cv2

warnings.filterwarnings("ignore")


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [6]:
# Window Length.
n = 100
STRIDE = 50

In [7]:
def getFeatures(data, fs):
    f1 = np.mean(data, axis=0)
    C = np.cov(data.T)
    f2 = np.concatenate((C[0, 0:3], C[1, 1:3], C[2, 2:3]))
    f3 = np.array([skew(data.iloc[:, 0]), skew(data.iloc[:, 1]), skew(data.iloc[:, 2])])
    f4 = np.array([kurtosis(data.iloc[:, 0]), kurtosis(data.iloc[:, 1]), kurtosis(data.iloc[:, 2])])
    f5 = np.zeros(3)
    f6 = np.zeros(3)
    for i in range(0,3):
        g = abs(np.fft.fft(data.iloc[:,i]))
        g = g[0:round(len(g)/2)]
        g[0] = 0
        max_i = np.argmax(g)
        f5[i] = g[max_i]
        f6[i] = fs * max_i
    return np.concatenate((f1, f2, f3, f4, f5, f6))

In [8]:
def generate_features(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[i for i in range(0, 42)])
    fs = 1 / n
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        features_accel = getFeatures(df.iloc[i:i+n, 0:3], fs)
        features_gyro = getFeatures(df.iloc[i:i+n, 3:6], fs)
        features = np.concatenate((features_accel, features_gyro))
        df_out.loc[i] = features
        i += 1
    return df_out

In [9]:
# Choose via MODE ## TBD ##
def get_detection_window(df, stride=STRIDE):
    df_out = pd.DataFrame(columns=[0])
    i = 0
    for i in range(0, df.shape[0] - n, stride):
        df_slice = df.iloc[i:i+n,:]
        df_out.loc[i] = df_slice.mode().iloc[0,0]
#         df_out.loc[i] = df.iloc[i+n-1]
        i += 1
    return df_out

In [10]:
def extrapolate_predictions(predictions, size):
    extrapolated = np.fromiter((val for val in predictions for _ in range(0, STRIDE)), float)
    return extrapolated[0:size]

In [11]:
def read_test_data(session):
    df_data_arm = pd.read_csv('test_data/{}/armIMU.txt'.format(session), delim_whitespace=True, header=None)
    print('debug_read_data_1')
    size = df_data_arm.shape[0]
    print('debug_read_data_2')
    arm_features = generate_features(df_data_arm, stride=1)
    print('debug_read_data_3')
    df_data_wrist = pd.read_csv('test_data/{}/wristIMU.txt'.format(session), delim_whitespace=True, header=None)
    print('debug_read_data_4')
    wrist_features = generate_features(df_data_wrist, stride=1)
    print('debug_read_data_5')
#     df_data_detection = pd.read_csv('test_data/{}/detection.txt', header=None)
#     df_data_detection = get_detection_window(df_data_detection, stride=1)
    zero_padding = pd.DataFrame(np.zeros(n))
    print('debug_read_data_6')
    return size, pd.concat([arm_features, wrist_features], axis=1, sort=False, ignore_index=True)

In [12]:
from keras.layers import Conv2DTranspose, Dropout, Activation, BatchNormalization, UpSampling2D, Concatenate
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate
from keras.models import Model

In [13]:
def conv1d_block(input_tensor, n_filters, kernel_size):
    # first layer
    # Create a Conv2D layer with n_filters and a kernel of dimension : kernel_size x kernel_size. 
    # Use same padding and he_normal initializer
    # YOUR CODE HERE
#     x = Conv2D(n_filters, (kernel_size, kernel_size), strides=(1, 1), padding='same', data_format=None, dilation_rate=(1, 1),
#         activation=None, use_bias=True, kernel_initializer='he_normal', bias_initializer='zeros', kernel_regularizer=None,
#         bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(input_tensor)
    x = Conv2D(n_filters, (kernel_size, kernel_size), kernel_initializer='he_normal', padding="same")(input_tensor)

    # add a BatchNormalization layer
    # YOUR CODE HERE
    x = BatchNormalization()(x)

        
    # Add a relu non-linearity (keras.layers.Activation)
    # YOUR CODE HERE
    x = Activation('relu')(x)
    
    # second layer
    # repeat the above steps (Conv + batchnorm + relu) taking the output of relu layer as input for this convolutional layer
    # YOUR CODE HERE
    x = Conv2D(n_filters, kernel_size = (kernel_size, kernel_size), strides=(1, 1), padding = 'same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # return the output tensor
    return x

In [ ]:
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv1D(...))
model.add(TimeDistributed(MaxPooling1D(...)))
model.add(TimeDistributed(Flatten()))
# define LSTM model
model.add(LSTM(...))
model.add(Dense(...))